In [1]:
import numpy as np
import open3d as o3d
from tqdm import tqdm_notebook
import os, sys, shutil

In [2]:
BASE_PATH = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/iclr_preprocess"

FOLDER = "d"
OUTPUT_FOLDER = "d_40_512"
IS_DYNAMIC = True

# STATIC_PATH  = os.path.join(BASE_PATH, STATIC_FOLDER)
FOLDER_PATH = os.path.join(BASE_PATH, FOLDER)
if IS_DYNAMIC:
    MASK_PATH = os.path.join(BASE_PATH, "m")
    if not os.path.exists(MASK_PATH):
        assert False
OUTPUT_FOLDER_PATH = os.path.join(BASE_PATH, OUTPUT_FOLDER)
if not os.path.exists(OUTPUT_FOLDER_PATH):
    os.makedirs(OUTPUT_FOLDER_PATH)
else:
    shutil.rmtree(OUTPUT_FOLDER_PATH)
    os.makedirs(OUTPUT_FOLDER_PATH)

In [3]:
def from_polar_np(velo):
    angles = np.linspace(0, np.pi * 2, velo.shape[-1])
    dist, z = velo[:, 0], velo[:, 1]
    x = np.cos(angles) * dist
    y = np.sin(angles) * dist
    out = np.stack([x,y,z], axis=1)
    return out.astype('float32')

def get_pcd_from_img(img):
    img = img * 120
    shp = img.shape
    img = img.reshape(1, shp[0], shp[1], shp[2])
    frame = from_polar_np(img)
    # frame_actual = np.array([frame_image[:29] for frame_image in frame])
    frame_flat = frame.reshape((3,-1))
    some_pcd = o3d.geometry.PointCloud()
    some_arr = frame_flat.T
    if IS_DYNAMIC:
        some_arr = some_arr[~np.array([pt[0]==0 and pt[1]==0 and pt[2]==0 for pt in some_arr])]
    some_pcd.points = o3d.utility.Vector3dVector(some_arr)
    return some_pcd

def draw_pcd(pcd, where='mat_3d'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.grid()
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.grid()
        plt.show()
        
def getint(name):
    try:
        return int(name.split('.')[0])
    except Exception as e:
        print("Error occured while trying to read {}".format(name))
    return None

In [4]:
file_idx = 1
for npy_file in sorted(os.listdir(FOLDER_PATH), key=getint):
    print(npy_file)
    npy_path = os.path.join(FOLDER_PATH, npy_file)
    npy_arr = np.load(npy_path)
    if IS_DYNAMIC:
        mask_file = os.path.join(MASK_PATH, npy_file)
        mask_arr = np.load(mask_file)
        new_mask = np.expand_dims(mask_arr[:,0], axis=1)
        new_mask_arr = np.concatenate([new_mask, new_mask], axis=1)
        npy_arr = npy_arr * new_mask_arr
        
    for arr in tqdm_notebook(npy_arr):
        this_pcd = get_pcd_from_img(arr[:,5:45,::2])
        
        pcd_file = str(file_idx) + ".ply"
        pcd_path = os.path.join(OUTPUT_FOLDER_PATH, pcd_file)
        o3d.io.write_point_cloud(pcd_path, this_pcd, write_ascii=True)
        
        file_idx += 1

0.npy


/home/saby/anaconda3/envs/ati/lib/python3.6/site-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]



1.npy



3.npy


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




11.npy


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




15.npy


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

